## Helper. Draft

In [1]:
import numpy as np
import pandas as pd

from collections import OrderedDict
import yaml

## Data

In [2]:
map_data = pd.read_csv('data/SHR65_22.csv', index_col=0)
map_data.head()

,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,Month,Incident,...,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,MSA
ID,,,,,,,,,,,,,,,,,,,,,
197601001AKASP00,"Juneau, AK",AKASP00,Alaska,State Troopers,Primary state LE,FBI,Yes,1976,January,1,...,American Indian or Alaskan Native,Unknown or not reported,Knife or cutting instrument,Husband,Other arguments,NaN,0,0,30180.0,Rural Alaska
197601001AL00102,"Jefferson, AL",AL00102,Alabama,Birmingham,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Shotgun,Acquaintance,Felon killed by private citizen,Felon killed in commission of a crime,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00104,"Jefferson, AL",AL00104,Alabama,Fairfield,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Shotgun,Wife,Other,NaN,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00106,"Jefferson, AL",AL00106,Alabama,Leeds,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Knife or cutting instrument,Brother,Other arguments,NaN,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00201,"Mobile, AL",AL00201,Alabama,Mobile,Municipal police,FBI,Yes,1976,January,1,...,White,Unknown or not reported,Strangulation - hanging,Acquaintance,Circumstances undetermined,NaN,0,0,30180.0,"Mobile, AL"


In [3]:
weapon_index = map_data.groupby('Weapon').count()['Ori'].index
weapon = pd.DataFrame(index=weapon_index, columns=['Solved', 'Not Solved', 'Ratio'])
weapon['Solved'] = map_data[map_data['Solved']=='Yes'].groupby('Weapon').count()['Ori']
weapon['Not Solved'] = map_data[map_data['Solved']=='No'].groupby('Weapon').count()['Ori']
weapon['Ratio'] = weapon['Solved'] / weapon['Not Solved']
weapon

,Solved,Not Solved,Ratio
Weapon,,,
Asphyxiation - includes death by gas,4027,1216,3.311678
"Blunt object - hammer, club, etc",25797,11082,2.327829
Drowning,1247,270,4.618519
Explosives,451,185,2.437838
Fire,5493,2596,2.115948
"Firearm, type not stated",46569,40264,1.156591
"Handgun - pistol, revolver, etc",289898,130956,2.213705
Knife or cutting instrument,97635,25447,3.836798
"Narcotics or drugs, sleeping pills",2934,481,6.099792


In [19]:
def start_message(state, year):
    message = '------------------------\n'
    if state:
        message += f'for {state.upper()}, '
    else:
        message += 'for all states. '
    if year:
        message += f'in {year}:'
    else:
        message += 'for all years.'
    
    message += '\n------------------------'
    return message    
    

def sk_info(state='', year=0):
    data = map_data.copy()
    if state:
        data = data[data['State']==state]
    if year:
        data = data[data['Year']==year]
    
    solved_or_not_info = OrderedDict()
    solved_or_not_info['all cases'] = data.shape[0]
    solved_or_not_info['victims count ???'] = int(data['VicCount'].sum())
    solved_or_not_info['solved cases'] = data[data['Solved']=='Yes'].shape[0]
    solved_or_not_info['not solved cases'] = solved_or_not_info['all cases'] - solved_or_not_info['solved cases']
    solved_or_not_info['ratio solved / not solved'] = solved_or_not_info['solved cases'] / solved_or_not_info['not solved cases']
    
    weapon_index = data.groupby('Weapon').count()['Ori'].index
    weapon = pd.DataFrame(index=weapon_index, columns=['Solved', 'Not Solved', 'Ratio'])
    weapon['Solved'] = data[data['Solved']=='Yes'].groupby('Weapon').count()['Ori']
    weapon['Not Solved'] = data[data['Solved']=='No'].groupby('Weapon').count()['Ori']
    weapon['Ratio'] = weapon['Solved'] / weapon['Not Solved']
    
    circumstance_index = data.groupby('Circumstance').count()['Ori'].index
    circumstance = pd.DataFrame(index=circumstance_index, columns=['Solved', 'Not Solved', 'Ratio'])
    circumstance['Solved'] = data[data['Solved']=='Yes'].groupby('Circumstance').count()['Ori']
    circumstance['Not Solved'] = data[data['Solved']=='No'].groupby('Circumstance').count()['Ori']
    circumstance['Ratio'] = circumstance['Solved'] / circumstance['Not Solved']
    
    vic_race_index = data.groupby('VicRace').count()['Ori'].index
    vic_race = pd.DataFrame(index=vic_race_index, columns=['Solved', 'Not Solved', 'Ratio'])
    vic_race['Solved'] = data[data['Solved']=='Yes'].groupby('VicRace').count()['Ori']
    vic_race['Not Solved'] = data[data['Solved']=='No'].groupby('VicRace').count()['Ori']
    vic_race['Ratio'] = vic_race['Solved'] / vic_race['Not Solved']
    
    
    print(start_message(state, year))
    print(yaml.dump(solved_or_not_info))
    print(weapon, '\n')
    print(circumstance, '\n')
    print(vic_race, '\n')


sk_info('Alabama', 1989)


------------------------
for ALABAMA, in 1989:
------------------------
!!python/object/apply:collections.OrderedDict
- - [all cases, 416]
  - ['victims count ???', 32]
  - [solved cases, 366]
  - [not solved cases, 50]
  - [ratio solved / not solved, 7.32]

                                    Solved  Not Solved      Ratio
Weapon                                                           
Blunt object - hammer, club, etc      11.0         3.0   3.666667
Drowning                               3.0         NaN        NaN
Explosives                             NaN         1.0        NaN
Fire                                   NaN         2.0        NaN
Firearm, type not stated               7.0         NaN        NaN
Handgun - pistol, revolver, etc      185.0        20.0   9.250000
Knife or cutting instrument           67.0        10.0   6.700000
Other or type unknown                  9.0         6.0   1.500000
Personal weapons, includes beating    12.0         2.0   6.000000
Rifle          